<a href="https://colab.research.google.com/github/marreapato/Deep_Learning_Course/blob/main/VLModel_Lucas_Rabelo_modelo_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando Modelo

In [ ]:
!pip install transformers!pip install transformers

In [21]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.9 MB/s eta 0:00:00


In [ ]:
import timm
timm.list_models()#vit_base_patch32_clip_224


In [24]:
model = timm.create_model('vit_base_patch32_clip_224', pretrained=True, num_classes=10)
x     = torch.randn(1, 3, 224, 224)
model(x).shape

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

torch.Size([1, 10])

In [25]:
model.forward_features(x).shape

torch.Size([1, 50, 768])

In [26]:
model.pretrained_cfg# architecture of the model matches chosen architeture in the beggining (torch architecture)

{'url': '',
 'hf_hub_id': 'timm/vit_base_patch32_clip_224.laion2b_ft_in12k_in1k',
 'architecture': 'vit_base_patch32_clip_224',
 'tag': 'laion2b_ft_in12k_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'fixed_input_size': True,
 'interpolation': 'bicubic',
 'crop_pct': 0.9,
 'crop_mode': 'center',
 'mean': (0.48145466, 0.4578275, 0.40821073),
 'std': (0.26862954, 0.26130258, 0.27577711),
 'num_classes': 1000,
 'pool_size': None,
 'first_conv': 'patch_embed.proj',
 'classifier': 'head'}

In [17]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [20]:
model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

#Connect with Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
%cd /content/gdrive/MyDrive/Fundamentals_Deep_Learning

!ls




/content/gdrive/MyDrive/Fundamentals_Deep_Learning
 001_tensorflow.ipynb		     BANKEX.csv
 00_jupyterlab.ipynb		    'Cópia de 01_mnist.ipynb'
 01_mnist.ipynb			     data
 02_asl.ipynb			     GRU_B20
 03_asl_cnn.ipynb		     images
 04a_asl_augmentation.ipynb	     LSTM_B20
 04b_asl_predictions.ipynb	     mit_indoor_scene_dataset
 05a_doggy_door.ipynb		    'NN FeedForward - CIFAR - 10.ipynb'
 05b_presidential_doggy_door.ipynb   original
 06_headline_generator.ipynb	     outdir
 activities.csv			     RNN_multi.ipynb
 asl_model


In [27]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For separating train and test sets
from sklearn.model_selection import train_test_split

# For visualizations
import matplotlib.pyplot as plt
import matplotlib.image as img
import PIL.Image as Image
import cv2

import os
import numpy as np
import pathlib

data_dir = "/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original"
data_dir = pathlib.Path(data_dir)

In [28]:
import torch
from torchvision import transforms, datasets

# Define the data transformation pipeline
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),     # Resize the image to (224, 224)
    transforms.ToTensor(),             # Convert image to tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ),
    ])

# Specify the root directory of your dataset
data_dir = "/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/train"
data_dir_test = "/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/test"

# Create an instance of the ImageFolder dataset
train_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=data_dir_test, transform=data_transform)

dataset_size = len(train_dataset)
indices = list(range(dataset_size))

In [29]:
train_dataset.class_to_idx

{'bird': 0,
 'car': 1,
 'cat': 2,
 'deer': 3,
 'dog': 4,
 'equine': 5,
 'frog': 6,
 'plane': 7,
 'ship': 8,
 'truck': 9}

In [30]:
len(test_dataset)

5286

In [31]:
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split


# Create a DataLoader to load data in batches
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    #sampler=train_sampler
    )

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    #sampler=test_sampler
    )

# Iterate through the DataLoader
#for inputs, classes in train_dataloader:
 #   print(inputs.shape, classes)
    #print(labels.min(), labels.max())
  #  break

In [32]:
import torch

torch.__version__

'2.1.0+cu118'

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm.notebook import tqdm
import torch.nn as nn
from torch import optim



In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm.notebook import tqdm

def evaluate(dataloader, model):
  model.cuda()
  model.eval()

  targets = []
  all_preds = []

  for x, y in tqdm(dataloader):
    logits = model(x.cuda())
    pred = logits.argmax(1).flatten().cpu().tolist()

    targets.extend(y.cpu().tolist())
    all_preds.extend(pred)

  acc = accuracy_score(y_true=targets, y_pred=all_preds)
  cm = confusion_matrix(y_true=targets, y_pred=all_preds)
  f1 = f1_score(y_true=targets, y_pred=all_preds, average='weighted')

  return acc, cm, f1

evaluate(test_dataloader, model)

  0%|          | 0/83 [00:00<?, ?it/s]

(0.07794173287930382,
 array([[  0,  16,   1,   2, 196,  97,   0,   7, 142,  72],
        [ 18,  71,  56,  11,  31,  35, 174,   5,  97,  36],
        [ 49,  73,   2,  41,   3,  35,  21,  31,  28, 249],
        [ 68,  38,  18,  27,  72, 164,  38,   5,  67,  33],
        [ 58,  11,  98,  15,  81,  47,  88,  11,  96,  26],
        [210,   2,   3,  11,  65,  37,  10, 127,  62,   6],
        [ 41,  20,  48,  12, 271,   9,  14,  34,   0,  81],
        [ 41,  21,  71,  25,  75, 152,   7, 117,  12,   8],
        [174,  19,   0,   0,  23,   0, 184,  77,  55,   0],
        [ 34,  31,  23, 126, 112, 108,   4,  51,   5,   8]]),
 0.07944905388194279)

In [35]:
evaluate(train_dataloader, model)

  0%|          | 0/331 [00:00<?, ?it/s]

(0.08455569224945585,
 array([[  2,  71,   5,   4, 677, 514,   3,  68, 546, 236],
        [ 58, 287, 154,  64,  88, 112, 857,   7, 381, 118],
        [233, 235,  10, 152,  14, 187,  98, 173, 157, 869],
        [139, 141,  54, 205, 289, 411, 317,  39, 318, 223],
        [207,  47, 269,  65, 313, 153, 385,  69, 557,  64],
        [819,   7,   6,  42, 228, 129,  74, 441, 350,  26],
        [145, 104, 203,  20, 931,  97,  44, 249,  31, 303],
        [107,  37, 270, 186, 334, 488,  50, 495,  92,  51],
        [495,  43,   3,   3,  50,   3, 713, 522, 277,  10],
        [ 97, 138,  78, 496, 413, 441,  11, 279,  33,  25]]),
 0.08468300437927462)

In [36]:
def epoch(dataloader, model, criterion, optimizer):
  model.cuda()
  model.train()

  epoch_loss = 0.0
  num_batches = 0

  for x, y in tqdm(dataloader):
    # erase the current model gradients
    optimizer.zero_grad()

    # forward
    logits = model(x.cuda())
    loss = criterion(input=logits, target=y.cuda())

    # backward
    loss.backward()

    # weights update ~ w = w_cur - lr*grad(loss, w)
    optimizer.step()

    epoch_loss += loss.detach().cpu().item()
    num_batches += 1

  return epoch_loss/num_batches

In [37]:
summary = {
    'model_state':None,
    'f1_history':[],
    'acc_history':[],
    'cm_history':[],
    'train_loss_history':[],
    'train_acc_history': []
}

lr = 1e-3
num_epochs = 10
best_acc = 0.0

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss(weight=None)


for e in range(num_epochs):
  print('Epoch:', e)
  cur_loss = epoch(train_dataloader, model, criterion, optimizer)

  # Calculate training accuracy
  train_acc, train_cm, train_f1 = evaluate(train_dataloader, model)#gets only train_acc

  test_acc, test_cm, test_f1 = evaluate(test_dataloader, model)

  summary['f1_history'].append(test_f1)
  summary['acc_history'].append(test_acc)
  summary['cm_history'].append(test_cm)
  summary['train_loss_history'].append(cur_loss)
  summary['train_acc_history'].append(train_acc)

  if test_acc > best_acc:
    summary['model_state'] = model.state_dict()
    best_acc = test_acc

  print('\t Training loss:', train_acc)
  print('\t Training accuracy:', test_f1)
  print('\t Test accuracy:', test_acc)
  print('\t Test confusion-matrix:', test_cm)
  print('\t Test F1:', test_f1)
  print()



Epoch: 0


  0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

	 Training loss: 0.2773729535345888
	 Training accuracy: 0.22479339447172497
	 Test accuracy: 0.2633371169125993
	 Test confusion-matrix: [[105  21  62 186   0  24  49  45  26  15]
 [ 57  51 126  29   0  21   2  73  96  79]
 [ 14  22 230 110   2  11   2  48  60  33]
 [ 65  11  44 325   0  14  21  25  14  11]
 [ 35  19 179 180   1  23   4  35  24  31]
 [ 84  28 132 179   0  39  15  18  19  19]
 [ 69  16  48 260   0  20  76  21  11   9]
 [ 28  14  41  28   1   7   1 252 118  39]
 [  5  11  23  14   0   2   3 207 239  28]
 [ 24  40  89  13   1  15   2 131 113  74]]
	 Test F1: 0.22479339447172497

Epoch: 1


...

	 Training loss: 0.29246711460206304
	 Training accuracy: 0.25274377759641714
	 Test accuracy: 0.2852818766553159
	 Test confusion-matrix: [[112   7  98  33  72   7 128  58  11   7]
 [ 18  38 195   4  56   9   5 109  56  44]
 [ 15  10 272  28  96   0  16  71  17   7]
 [ 45  10  62 111 103   2 135  49  10   3]
 [ 19  12 184  41 163   4  31  57  12   8]
 [ 44  30 128  68 142  15  39  45  10  12]
 [ 47  12  87  36  82  10 211  31   9   5]
 [  5   9  31  13  17   0   2 331 106  15]
 [  1   6  37   4   7   0   9 224 226  18]
 [  7  29 130   3  33   2   4 157 108  29]]
	 Test F1: 0.25274377759641714

Epoch: 9


  0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

	 Training loss: 0.30751395855020347
	 Training accuracy: 0.28143597732649583
	 Test accuracy: 0.3015512674990541
	 Test confusion-matrix: [[216  10  46  44  51   6  51  59  15  35]
 [ 44  71  52  11  33  20   2  57 135 109]
 [ 30  34 156  34 120   5  10  43  44  56]
 [ 98  15  39 166  94   7  53  30  12  16]
 [ 61  29 104  49 168   5   8  30  28  49]
 [112  34  65  87  89  31  19  32  24  40]
 [169  16  42  74  63   7 105  26  13  15]
 [ 13  17  17  14  11   4   2 267 152  32]
 [  7  13  10   8   9   1   3 145 311  25]
 [ 25  43  28   7  15   3   1 101 176 103]]
	 Test F1: 0.28143597732649583



In [38]:
summary_path = '/content/gdrive/MyDrive/Fundamentals_Deep_Learning/original/modelo_clip.pth'

torch.save(summary, summary_path)